In [1]:
import sqlite3
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import openai
import warnings
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [2]:
openai.api_key = os.environ.get("MY_OPENAI_API_KEY")
EMBED = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=openai.api_key)
DB_NAME = "test_vector_db"
DB_PATH = "/Users/yeongung/3355/main/ReadRiddle/chroma"

data = []
# SQLite 데이터베이스 연결 (파일명: db.sqlite3)
conn = sqlite3.connect("/Users/yeongung/3355/main/ReadRiddle/db.sqlite3")

# 커서 생성
cur = conn.cursor()

# 조건부 조회 (category가 "PYTHON"인 데이터 필터링)
category_value = "React"  # 검색할 카테고리 값
cur.execute("SELECT * FROM chatbot_documents WHERE title = ?", (category_value,))

# 결과 가져오기
filtered_rows = cur.fetchall()

# 출력
for row in filtered_rows:
    data.append(row[2])

# 연결 닫기
cur.close()
conn.close()

CHUNK_SIZE = 2000
CHUNK_OVERLAP = 200

docs = []

def split_texts(texts, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    documents = [Document(page_content=texts)]
    return recursive_text_splitter.split_documents(documents)

seen_texts = set()

for doc in data:
    # 이미 같은 내용이 있다면 제외
    if doc not in seen_texts:
        seen_texts.add(doc)
        docs += split_texts(doc)

In [3]:
len(data)

131

In [4]:
docs

[Document(metadata={}, page_content="Built-in React Hooks – React React v 19 Search ⌘ Ctrl K Learn Reference Community Blog  Overview Hooks useActionState useCallback useContext useDebugValue useDeferredValue useEffect useId useImperativeHandle useInsertionEffect useLayoutEffect useMemo useOptimistic useReducer useRef useState useSyncExternalStore useTransition Components <Fragment> (<>) <Profiler> <StrictMode> <Suspense> APIs act cache createContext lazy memo startTransition use experimental_taintObjectReference - This feature is available in the latest Canary experimental_taintUniqueValue - This feature is available in the latest Canary  Hooks useFormStatus Components Common (e.g. <div>) <form> <input> <option> <progress> <select> <textarea> <link> <meta> <script> <style> <title> APIs createPortal flushSync preconnect prefetchDNS preinit preinitModule preload preloadModule Client APIs createRoot hydrateRoot Server APIs renderToPipeableStream renderToReadableStream renderToStaticMarku

In [5]:
len(docs)

1051

In [6]:
text = docs[0:10]

In [7]:
text

[Document(metadata={}, page_content="Built-in React Hooks – React React v 19 Search ⌘ Ctrl K Learn Reference Community Blog  Overview Hooks useActionState useCallback useContext useDebugValue useDeferredValue useEffect useId useImperativeHandle useInsertionEffect useLayoutEffect useMemo useOptimistic useReducer useRef useState useSyncExternalStore useTransition Components <Fragment> (<>) <Profiler> <StrictMode> <Suspense> APIs act cache createContext lazy memo startTransition use experimental_taintObjectReference - This feature is available in the latest Canary experimental_taintUniqueValue - This feature is available in the latest Canary  Hooks useFormStatus Components Common (e.g. <div>) <form> <input> <option> <progress> <select> <textarea> <link> <meta> <script> <style> <title> APIs createPortal flushSync preconnect prefetchDNS preinit preinitModule preload preloadModule Client APIs createRoot hydrateRoot Server APIs renderToPipeableStream renderToReadableStream renderToStaticMarku

In [8]:
def create_vstore(DOCS, DB_NAME: str):

    return Chroma.from_documents(
        documents=DOCS,
        collection_name=DB_NAME,
        embedding=EMBED,
    )

In [9]:
vectorstore = FAISS.from_documents(documents=docs, embedding=EMBED)

In [10]:
vectorstore.save_local("/Users/yeongung/3355/main/ReadRiddle/chatbot/VDB/React/faiss_index")

In [11]:
new_vector_store = FAISS.load_local("/Users/yeongung/3355/main/ReadRiddle/chatbot/VDB/React/faiss_index", EMBED, allow_dangerous_deserialization=True)

In [12]:
retriever = new_vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 50}
)

In [13]:
retriever.invoke('cache')

[Document(id='8d2aea69-4a3d-4626-806f-eef408b21066', metadata={}, page_content="cache – React React v 19 Search ⌘ Ctrl K Learn Reference Community Blog  Overview Hooks useActionState useCallback useContext useDebugValue useDeferredValue useEffect useId useImperativeHandle useInsertionEffect useLayoutEffect useMemo useOptimistic useReducer useRef useState useSyncExternalStore useTransition Components <Fragment> (<>) <Profiler> <StrictMode> <Suspense> APIs act cache createContext lazy memo startTransition use experimental_taintObjectReference - This feature is available in the latest Canary experimental_taintUniqueValue - This feature is available in the latest Canary  Hooks useFormStatus Components Common (e.g. <div>) <form> <input> <option> <progress> <select> <textarea> <link> <meta> <script> <style> <title> APIs createPortal flushSync preconnect prefetchDNS preinit preinitModule preload preloadModule Client APIs createRoot hydrateRoot Server APIs renderToPipeableStream renderToReadab

In [14]:
from langchain_community.retrievers import BM25Retriever

In [15]:
retriever = BM25Retriever.from_documents(docs, embedding=EMBED)

In [16]:
retriever.invoke('cache')

[Document(metadata={}, page_content="it with the same arguments Reference cache(fn) Call cache outside of any components to create a version of the function with caching. <code_snipet> import {cache} from 'react';import calculateMetrics from 'lib/metrics';const getMetrics = cache(calculateMetrics);function Chart({data}) { const report = getMetrics(data); // ...} </code_snipet> When getMetrics is first called with data , getMetrics will call calculateMetrics(data) and store the result in cache. If getMetrics is called again with the same data , it will return the cached result instead of calling calculateMetrics(data) again. See more examples below. Parameters fn : The function you want to cache results for. fn can take any arguments and return any value. Returns cache returns a cached version of fn with the same type signature. It does not call fn in the process. When calling cachedFn with given arguments, it first checks if a cached result exists in the cache. If a cached result exist

In [17]:
import pickle
with open('/Users/yeongung/3355/main/ReadRiddle/chatbot/VDB/React/bm25_retriever.pkl', 'wb') as f:
    pickle.dump(retriever, f)

In [18]:
with open('/Users/yeongung/3355/main/ReadRiddle/chatbot/VDB/React/bm25_retriever.pkl', 'rb') as f:
    retriever = pickle.load(f)

In [19]:
retriever.invoke('cache')

[Document(metadata={}, page_content="it with the same arguments Reference cache(fn) Call cache outside of any components to create a version of the function with caching. <code_snipet> import {cache} from 'react';import calculateMetrics from 'lib/metrics';const getMetrics = cache(calculateMetrics);function Chart({data}) { const report = getMetrics(data); // ...} </code_snipet> When getMetrics is first called with data , getMetrics will call calculateMetrics(data) and store the result in cache. If getMetrics is called again with the same data , it will return the cached result instead of calling calculateMetrics(data) again. See more examples below. Parameters fn : The function you want to cache results for. fn can take any arguments and return any value. Returns cache returns a cached version of fn with the same type signature. It does not call fn in the process. When calling cachedFn with given arguments, it first checks if a cached result exists in the cache. If a cached result exist

In [20]:
print(
  """Django 모델을 작성하는 기본적인 예제는 다음과 같습니다:\n\n```python\nfrom django.db import models\n\nclass Person(models.Model):\n    first_name = models.CharField(max_length=30)\n    last_name = models.CharField(max_length=30)\n```\n\n위의 `Person` 모델은 `first_name`과 `last_name`이라는 두 개의 필드를 가지고 있습니다. 각 필드는 클래스 속성으로 정의되며, 각 속성은 데이터베이스 열에 매핑됩니다. 이 모델은 데이터베이스에 `myapp_person`이라는 테이블을 생성합니다.\n\n이 정보는 \"Models\" 문서에서 가져온 것입니다. (출처: Document id='572d848e-04f7-41e1-9e5a-0e9271754e07')"""
  
)

Django 모델을 작성하는 기본적인 예제는 다음과 같습니다:

```python
from django.db import models

class Person(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)
```

위의 `Person` 모델은 `first_name`과 `last_name`이라는 두 개의 필드를 가지고 있습니다. 각 필드는 클래스 속성으로 정의되며, 각 속성은 데이터베이스 열에 매핑됩니다. 이 모델은 데이터베이스에 `myapp_person`이라는 테이블을 생성합니다.

이 정보는 "Models" 문서에서 가져온 것입니다. (출처: Document id='572d848e-04f7-41e1-9e5a-0e9271754e07')
